In [1]:
#First we have to import findspark and provide the location of our spark executor.
import findspark
findspark.init("/Users/martin/Downloads/spark-3.1.2-bin-hadoop3.2")
findspark.find

<function findspark.find()>

In [2]:
#Import all necessary components
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark

In [3]:
#Configurate Spark parameters
spark = SparkSession.builder \
.master("local") \
.appName("Sparky") \
.config("spark.executor.memory", "1gb") \
.getOrCreate()

sc = spark.sparkContext

In [4]:
#Load the titanic.csv file into a spark dataframe and cast the values to the approprate types. Then show the dataframe

df = spark.read.format("CSV").option("header", "true").load("titanic.csv")

df = df.withColumn("Survived", df["Survived"].cast(IntegerType()))
df = df.withColumn("Pclass", df["Pclass"].cast(IntegerType()))
df = df.withColumn("Name", df["Name"].cast(StringType()))
df = df.withColumn("Sex", df["Sex"].cast(StringType()))
df = df.withColumn("Age", df["Age"].cast(IntegerType()))
df = df.withColumn("Siblings/Spouses Aboard", df["Siblings/Spouses Aboard"].cast(IntegerType()))
df = df.withColumn("Parents/Children Aboard", df["Parents/Children Aboard"].cast(IntegerType()))
df = df.withColumn("Fare", df["Fare"].cast(FloatType()))

df.show()

+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|Survived|Pclass|                Name|   Sex|Age|Siblings/Spouses Aboard|Parents/Children Aboard|   Fare|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|       0|     3|Mr. Owen Harris B...|  male| 22|                      1|                      0|   7.25|
|       1|     1|Mrs. John Bradley...|female| 38|                      1|                      0|71.2833|
|       1|     3|Miss. Laina Heikk...|female| 26|                      0|                      0|  7.925|
|       1|     1|Mrs. Jacques Heat...|female| 35|                      1|                      0|   53.1|
|       0|     3|Mr. William Henry...|  male| 35|                      0|                      0|   8.05|
|       0|     3|     Mr. James Moran|  male| 27|                      0|                      0| 8.4583|
|       0|     1|Mr. Timothy J McC...|  male| 

In [5]:
# Assignment 3A

#First we select the needed columns (Survived, class, sex) and group them by class and sex.
#Then we calculate the average survival rate and show the columns with the appropriate names.
df_survival_rate_class_sex = df.select("Survived", "Pclass", "Sex").groupBy("Pclass", "Sex").avg("Survived").toDF("Pclass", "Sex", "Survival Percent")
df_survival_rate_class_sex.show()

+------+------+-------------------+
|Pclass|   Sex|   Survival Percent|
+------+------+-------------------+
|     2|female| 0.9210526315789473|
|     3|  male|0.13702623906705538|
|     1|  male|0.36885245901639346|
|     3|female|                0.5|
|     1|female| 0.9680851063829787|
|     2|  male| 0.1574074074074074|
+------+------+-------------------+



In [7]:
#Assignment 3B

#First we select the survived, class, age columns and filter them, so that we only have 3-rd class children under 10 y/o.
df_third_class_children = df.select("Survived", "Pclass", "Age").filter((df.Pclass == 3) & (df.Age <= 10))

#Next we see the survival rate of the filtered passengers
survived = df_third_class_children.filter(df_third_class_children.Survived == 1).count()

#Now we calculate the survival rate and cast it to string so that we can print it out
survival_rate = str(survived/df_third_class_children.count()*100)

print ("The survival rate of 3-rd class children under 10 years old is: " + survival_rate + " %")

The survival rate of 3-rd class children under 10 years old is: 41.509433962264154 %


In [8]:
#3C

#First we select the class and fare from the dataframe and group them by the class and fare. 
#Then we calculate the average fares to see the expected price per class 
df_expected_fare = df.select("Pclass", "Fare").groupBy("Pclass").avg("Fare").toDF("class", "Expected fare")
df_expected_fare.show()

+-----+------------------+
|class|     Expected fare|
+-----+------------------+
|    1| 84.15468752825701|
|    3|13.707707501045244|
|    2| 20.66218318109927|
+-----+------------------+



In [9]:
#We have completed our task so we close spark.
spark.stop()